In [1]:
from dscribe.descriptors import SOAP

species = ["H", "C", "O", "N"]
# 局所領域のカットオフ（オングストローム単位）。1オングストロームより大きくする必要があります(6.0)
r_cut = 6.0
# ラジアル基底関数の数(8)
n_max = 2
# 球面調和関数の最大次数(2)
l_max = 0

# Setting up the SOAP descriptor
soap = SOAP(
    species=species,
    periodic=False,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
)

# Creation
from ase.build import molecule

# Molecule created as an ASE.Atoms
water = molecule("H2O")

# Create SOAP output for the system
soap_water = soap.create(water, centers=[0])

# 出力はthe partial power spectrum vector 𝐩(𝐫)　(部分パワースペクトルベクトル)
print(soap_water)
print(soap.get_number_of_features())
print(soap_water.shape)
print(type(soap_water))

[[  2.07826909  15.06330421 109.17890023   0.           0.
    0.           0.           0.           0.           0.
    0.           2.15371645   8.56656642  15.61014703  62.09051397
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           2.23190277   8.8775583
   35.31114453]]
36
(1, 36)
<class 'numpy.ndarray'>


In [2]:
# Create output for multiple system
samples = [molecule("H2O"), molecule("NO2"), molecule("CO2")]
centers = [[0], [1, 2], [1, 2]]
coulomb_matrices = soap.create(samples, centers)            # Serial
coulomb_matrices = soap.create(samples, centers, n_jobs=2)  # Parallel

# Lets change the SOAP setup and see how the number of features changes
small_soap = SOAP(species=species, r_cut=r_cut, n_max=2, l_max=0)
big_soap = SOAP(species=species, r_cut=r_cut, n_max=9, l_max=9)
n_feat1 = small_soap.get_number_of_features()
n_feat2 = big_soap.get_number_of_features()
print(n_feat1, n_feat2)

36 6660


In [ ]:
# Periodic systems
from ase.build import bulk

copper = bulk('Cu', 'fcc', a=3.6, cubic=True)
print(copper.get_pbc())
periodic_soap = SOAP(
    species=[29],
    r_cut=r_cut,
    n_max=n_max,
    l_max=n_max,
    periodic=True,
    sparse=False
)

soap_copper = periodic_soap.create(copper)
print(soap_copper)
print(soap_copper.sum(axis=1))

[ True  True  True]
[[ 3.16969900e+01 -3.10589148e+02  3.04336844e+03  3.08805278e-31
  -3.79711084e-30  4.71194535e-29  6.18507922e-32 -1.51180520e-30
   3.81806855e-29]
 [ 3.16969900e+01 -3.10589148e+02  3.04336844e+03  1.88632783e-31
  -2.44485509e-30  3.29051018e-29  6.17456162e-32 -1.37591476e-30
   3.24869288e-29]
 [ 3.16969900e+01 -3.10589148e+02  3.04336844e+03  1.22099999e-31
  -1.94613540e-30  3.10760577e-29  1.24765412e-32 -3.15215554e-31
   7.99721559e-30]
 [ 3.16969900e+01 -3.10589148e+02  3.04336844e+03  5.08229381e-32
  -7.03758175e-31  1.08181673e-29  5.74942715e-32 -1.53404715e-30
   4.09988087e-29]]
[2764.4762848 2764.4762848 2764.4762848 2764.4762848]


In [4]:
# Locations
# The locations of specific element combinations can be retrieved like this.
hh_loc = soap.get_location(("H", "H"))
ho_loc = soap.get_location(("H", "O"))

# These locations can be directly used to slice the corresponding part from an
# SOAP output for e.g. plotting.
soap_water[0, hh_loc]
soap_water[0, ho_loc]

# Sparse output
soap = SOAP(
    species=species,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    sparse=True
)
soap_water = soap.create(water)
print(soap_water)
print(type(soap_water))

soap = SOAP(
    species=species,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    sparse=False
)
soap_water = soap.create(water)
print(soap_water)
print(type(soap_water))

<COO: shape=(3, 36), dtype=float64, nnz=30, fill_value=0.0>
<class 'sparse._coo.core.COO'>
[[  2.07826909  15.06330421 109.17890023   0.           0.
    0.           0.           0.           0.           0.
    0.           2.15371645   8.56656642  15.61014703  62.09051397
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           2.23190277   8.8775583
   35.31114453]
 [  2.5355939   16.33039202 105.17524271   0.           0.
    0.           0.           0.           0.           0.
    0.           1.14778552   8.3191549    7.39226718  53.5791875
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.51956727   3.76582605
   27.29472506]
 [  2.5355939   16.33

In [5]:
# Average output
average_soap = SOAP(
    species=species,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
    sparse=False
)

soap_water = average_soap.create(water)
print("Average SOAP water: ", soap_water.shape)

methanol = molecule('CH3OH')
soap_methanol = average_soap.create(methanol)
print("Average SOAP methanol: ", soap_methanol.shape)

h2o2 = molecule('H2O2')
soap_peroxide = average_soap.create(h2o2)
print("Average SOAP peroxide: ", soap_peroxide.shape)

Average SOAP water:  (36,)
Average SOAP methanol:  (36,)
Average SOAP peroxide:  (36,)


In [6]:
# Distance
from scipy.spatial.distance import pdist, squareform
import numpy as np

molecules = np.vstack([soap_water, soap_methanol, soap_peroxide])
distance = squareform(pdist(molecules))
print("Distance matrix: water - methanol - peroxide: ")
print(distance)

Distance matrix: water - methanol - peroxide: 
[[  0.         170.82575231  75.47146683]
 [170.82575231   0.         211.49008614]
 [ 75.47146683 211.49008614   0.        ]]
